# Presentaion of Our Methods
The topic of our project is to design a metric to evaluate teh quality of summaries given the original text. After discussion and paper reading, we think that a good metric for this task should have the following properties:
1. It should be able to tell a good summary from a bad one (The scores of them should be as different as possible).
2. It should be able to discern varying degrees of factual distortion (Given any two summaries according to the same document, the worse one should be scored lower).
3. It should be able to make evaluation based on the detail of the summary (Give the reason why it makes such an evaluation result).

In this notebook, we will present our methods for the project and the results we obtained. The presentation will be divided into following parts, each part will cover the above three properties:
1. Results of baseline metrics
2. Results of our methods
3. Comparison of our methods with baseline metrics
4. Conclusion

In [1]:
# Importing libraries
import pandas as pd
from pipeline import SummaryGrader, NER_comparison, highlight

## 1. Results of baseline metrics
TO DO, (introdece the metrics we used as baseline, and the results we obtained.)

In [ ]:
# TO DO

## 2. Results of our methods
We designed two methods to evaluate the quality of summaries.
1. Named entity comparison: we compare the named entities in the summary with the named entities in the original text by `NER_comparison`. It will calculate two ratios, one is the ratio of named entities in the summary that are also in the original text, the other is the ratio of named entities in the original text that are also in the summary. In addition, It also provides a method `.comparison_display()` to highlight the named entities in the summary that are not in the original text, or the named entities in the original text that are not in the summary. This will help users to find the details of the result.
2. Summary grading based on sentence-level checking: we apply LLMs to help us check the consistency between the summary and the original text sentence by sentence through `SummaryGrader`. Its `.process()` method can return the ratio of sentences in the summary that are thought to be consistent with the original text and the list of indices of sentences in the summary that are thought to be inconsistent with the original text. In addition, we can use `highlight()` function to highlight the sentences in the summary that are thought to be inconsistent with the original text. This will help users to find the details of the result.

### 2.1 Named entity comparison

In [ ]:
# TO DO just show how to use the class

### 2.2 Summary grading based on sentence-level checking

In [ ]:
# TO DO show the result of the class from three properties above

## 3. Comparison of our methods with baseline metrics
In this section, we will compare our methods with the baseline metrics we used in the first section from the three properties we mentioned at the beginning. We will use the same dataset and the same summaries as in the first section. The results are as follows:

In [ ]:
# TO DO

## 4. Conclusion
TO DO